# Analysing IRA tweets

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import re
import matplotlib.pyplot as plt
import glob

import findspark
findspark.init()

from pyspark import SparkContext

from pyspark.sql import *
from pyspark.sql.functions import to_timestamp
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import explode
from pyspark.sql.functions import col, when, length

%matplotlib inline

spark = SparkSession.builder.getOrCreate()

In [2]:
# Set up data directory
DATA_DIR = 'data/'

## 1. Getting to know our main dataset

## TODO: Should we keep track of which tweet is Iranian and which one is Russian?

### Reading the data

In [3]:
tweets_text_df = spark.read.format("csv").option("header", "true").load(DATA_DIR+'*_troll_tweet_text.csv')
tweets_stats_df = spark.read.format("csv").option("header", "true").load(DATA_DIR+'*_troll_tweet_stats.csv')
tweets_meta_df = spark.read.format("csv").option("header", "true").load(DATA_DIR+'*_troll_tweet_metadata.csv')
tweets_user_df = spark.read.format("csv").option("header", "true").load(DATA_DIR+'*_troll_user.csv')

In [4]:
print((tweets_text_df.count(), len(tweets_text_df.columns)))
print((tweets_stats_df.count(), len(tweets_stats_df.columns)))
print((tweets_meta_df.count(), len(tweets_meta_df.columns)))
print((tweets_user_df.count(), len(tweets_user_df.columns)))

(10164244, 3)
(10164244, 17)
(10164244, 6)
(4327, 11)


### tweets_text

In [5]:
print((tweets_text_df.count(), len(tweets_text_df.columns)))
tweets_text_df.show(10)

(10164244, 3)
+------------------+--------------+--------------------+
|           tweetid|tweet_language|          tweet_text|
+------------------+--------------+--------------------+
|877919995476496385|            ru|"RT @ruopentwit: ...|
|492388766930444288|            ru|Серебром отколоко...|
|719455077589721089|            bg|@kpru С-300 в Ира...|
|536179342423105537|            ru|Предлагаю судить ...|
|841410788409630720|            bg|Предостережение а...|
|834365760776630272|            ru|Двойная утопия, и...|
|577490527299457024|            ru|RT @harkovnews: Н...|
|596522755379560448|            ru|RT @NovostiNsk: «...|
|567357519547207680|            en|As sun and cloud ...|
|665533117369876480|            ru|RT @vesti_news: Ш...|
+------------------+--------------+--------------------+
only showing top 10 rows



Unknown *tweet_language* can take both the value 'und', or null. We harmonize this column by setting all NaN to 'und'.

In [6]:
tweets_text_df = tweets_text_df.fillna('und',['tweet_language'])

### tweets_stats

In [7]:
print((tweets_stats_df.count(), len(tweets_stats_df.columns)))
tweets_stats_df.dtypes

(10164244, 17)


[('tweetid', 'string'),
 ('userid', 'string'),
 ('tweet_time', 'string'),
 ('in_reply_to_tweetid', 'string'),
 ('in_reply_to_userid', 'string'),
 ('quoted_tweet_tweetid', 'string'),
 ('is_retweet', 'string'),
 ('retweet_userid', 'string'),
 ('retweet_tweetid', 'string'),
 ('quote_count', 'string'),
 ('reply_count', 'string'),
 ('like_count', 'string'),
 ('retweet_count', 'string'),
 ('hashtags', 'string'),
 ('urls', 'string'),
 ('user_mentions', 'string'),
 ('poll_choices', 'string')]

We first convert the *tweet_time* into Datetime for ease of use, and we cast some columns into integers. We also create a static sql view of the main dataframe on which we can apply our SQL queries.

In [8]:
tweets_stats_df = tweets_stats_df.withColumn('tweet_time', to_timestamp(tweets_stats_df.tweet_time))
tweets_stats_df = tweets_stats_df.withColumn('quote_count', tweets_stats_df.quote_count.cast('int'))
tweets_stats_df = tweets_stats_df.withColumn('reply_count', tweets_stats_df.reply_count.cast('int'))
tweets_stats_df = tweets_stats_df.withColumn('like_count', tweets_stats_df.like_count.cast('int'))
tweets_stats_df = tweets_stats_df.withColumn('retweet_count', tweets_stats_df.retweet_count.cast('int'))
tweets_stats_df.createOrReplaceTempView("tweets_stats_sql")

We can now start splitting the data into smaller dataframes and remove the useless columns for each of those:
* **retweets_df** contains all the posts that are retweets.
* **replies_df** contains all the posts that are replies to other tweets.
* **normal_tweets_df** contains all the other ('normal') posts.

**NB:** some tweets have a value for *in_reply_to_userid* while their *in_reply_to_tweetid* is null (however the inverse never happens). Those are either replies to deleted tweets, or mentions of other users that were treated as replies. We decided to consider them as normal tweets.

In [9]:
# RETWEETS
retweets_df = spark.sql("SELECT * FROM tweets_stats_sql WHERE is_retweet=True")

# To understand how we selected the columns to remove, uncomment the next two lines
#for col in retweets_df:
#    retweets_df.select(col).distinct().show(10)

retweets_df = retweets_df.drop('in_reply_to_tweetid', 'in_reply_to_userid', 'is_retweet', 'quote_count', 'reply_count', 'like_count', 'retweet_count', 'poll_choices')
print((retweets_df.count(), len(retweets_df.columns)))

(3565521, 9)


In [10]:
# REPLIES
replies_df = spark.sql("SELECT * FROM tweets_stats_sql WHERE is_retweet=False AND in_reply_to_tweetid IS NOT NULL")
replies_df = replies_df.drop('retweet_tweetid', 'retweet_userid', 'is_retweet')
print((replies_df.count(), len(replies_df.columns)))

(605558, 14)


In [11]:
# NORMAL
normal_tweets_df = spark.sql("SELECT * FROM tweets_stats_sql WHERE is_retweet=False AND in_reply_to_tweetid IS NULL")
normal_tweets_df = normal_tweets_df.drop('in_reply_to_tweetid', 'retweet_tweetid', 'retweet_userid', 'is_retweet')
print((normal_tweets_df.count(), len(normal_tweets_df.columns)))

(5993165, 13)


We verify that the number of rows correspond and that we did not duplicate or remove any by accident.

In [12]:
print(tweets_stats_df.count(), retweets_df.count()+normal_tweets_df.count()+replies_df.count())

10164244 10164244


### tweets_meta

In [13]:
print((tweets_meta_df.count(), len(tweets_meta_df.columns)))
tweets_meta_df.show(10)

(10164244, 6)
+------------------+--------------+---------------+--------+---------+------------------+
|           tweetid|follower_count|following_count|latitude|longitude| tweet_client_name|
+------------------+--------------+---------------+--------+---------+------------------+
|849295393867399169|          4042|           1470|    null|     null|Twitter Web Client|
|567280957913587713|           272|            390|    null|     null|          iziaslav|
|493095247690612736|            89|            223|    null|     null|          vavilonX|
|493892174069903360|            89|            223|    null|     null|          vavilonX|
|512503798506721280|            89|            223|    null|     null|          vavilonX|
|499624206246871041|            89|            223|    null|     null|          vavilonX|
|491828568251707392|            89|            223|    null|     null|          vavilonX|
|493768356810731520|            89|            223|    null|     null|          vavilo

In [14]:
tweets_meta_df.createOrReplaceTempView("tweets_meta_sql")

It appears that the number of rows with a non-null *latitude*/*longitude* combination is very small compared to the size of dataset (less than 0.05%). Furthermore, several of them are repeated. We thus consider it rather useless and prefer dropping it.

In [15]:
temp = spark.sql("SELECT * FROM tweets_meta_sql WHERE latitude IS NOT NULL")
print(tweets_meta_df.count(), temp.count(), temp.select('latitude', 'longitude').distinct().count())

tweets_meta_df = tweets_meta_df.drop('latitude', 'longitude')
tweets_meta_df.createOrReplaceTempView("tweets_meta_sql")

10164244 4811 2958


The main feature that we can use to split the data here is *tweet_client_name*. When we take a closer look to this column, we discover that there are more than 400 values registered. Many of them seem unidentifiable.

However, we can see that a good amount of tweets are sent through official Twitter applications:
* **Twitter Web Client** accounts for around one third of the tweets in the dataset.
* **TweetDeck**, which allows to manage multiple accounts simultaneously, handles around 7% of the tweets.
* **Twitter For Android** is also in the top 15 applications used for those tweets.

Most of the other tweets are generated through automated social media managers, such as **twitterfeed** (which shut down in 2016), **dlvr.it**, or even **IFTTT**.

In [16]:
temp = spark.sql("SELECT tweet_client_name, COUNT(*) AS count FROM tweets_meta_sql GROUP BY tweet_client_name ORDER BY count DESC")
print(temp.count())
temp.show(15)

410
+-------------------+-------+
|  tweet_client_name|  count|
+-------------------+-------+
| Twitter Web Client|3275944|
|        twitterfeed|1472548|
|          TweetDeck| 708585|
|      newtwittersky| 393074|
|          bronislav| 308516|
|              IFTTT| 300152|
|           iziaslav| 299963|
|          rostislav| 289475|
|        generationπ| 285503|
|         Twibble.io| 268402|
|    Ohwee Messanger| 240051|
|Twitter for Android| 225275|
|            dlvr.it| 224024|
|NovaPress Publisher| 204583|
|Приложение для тебя| 159588|
+-------------------+-------+
only showing top 15 rows



In [17]:
temp = spark.sql("SELECT tweet_client_name, COUNT(*) AS count FROM tweets_meta_sql WHERE tweet_client_name LIKE '%Twitter%' GROUP BY tweet_client_name ORDER BY count DESC")
print(temp.count())
temp.show()

20
+--------------------+-------+
|   tweet_client_name|  count|
+--------------------+-------+
|  Twitter Web Client|3275944|
| Twitter for Android| 225275|
|  Twitter for iPhone|  60141|
|        Twitter Lite|  26199|
|Twitter for Andro...|  22190|
|Twitter for Websites|  14706|
|    Twitter for iPad|   5572|
|Twitter for  Android|   3642|
|      Twitter Nation|    813|
|Twitter for Nokia...|    442|
| Twitter for Windows|    133|
|Twitter for Black...|    126|
|Twitter for Windo...|     87|
|Twitter for Black...|     71|
|Twitterrific for iOS|     21|
|         Twitter Ads|     12|
|Twitter Business ...|      2|
|         Twitterfall|      1|
|Unfollow Tools fo...|      1|
|Twitter.com inc     |      1|
+--------------------+-------+



In the end, seeing how messy this dataset is, and how few (small) columns it has, we decided to not split it.

In [18]:
print((tweets_meta_df.count(), len(tweets_meta_df.columns)))

(10164244, 4)


### tweets_user

In [19]:
print((tweets_user_df.count(), len(tweets_user_df.columns)))
print(tweets_user_df.columns)
#tweets_user_df.show(10)

(4327, 11)
['userid', 'user_display_name', 'user_screen_name', 'user_reported_location', 'user_profile_description', 'user_profile_url', 'account_creation_date', 'account_language', 'follower_count', 'following_count', 'last_tweet_at']


We first convert the dates and integers present in the dataframe. This also treats the wrong encodings in those columns (such as a language ('en') present in *last_tweet_at*.

In [20]:
tweets_user_df = tweets_user_df.withColumn('account_creation_date', to_timestamp(tweets_user_df.account_creation_date))
tweets_user_df = tweets_user_df.withColumn('last_tweet_at', to_timestamp(tweets_user_df.last_tweet_at))
tweets_user_df = tweets_user_df.withColumn('follower_count', tweets_user_df.follower_count.cast('int'))
tweets_user_df = tweets_user_df.withColumn('following_count', tweets_user_df.following_count.cast('int'))
tweets_user_df.createOrReplaceTempView("tweets_user_sql")

There also appear to be some wrong encodings in *account_language*. All languages are represented by a two letters code (except for *en-gb* and *zh-cn*, which correspond respectively to British English and Mainland Chinese). But a very small number of rows contain a date or a text as language.

After looking further into that, we discovered that those accounts wrote tweets in many different languages. As it is impossible for us to determine which one is their preferred language, we decided to set those inconsistent values to *'und'*.

In [21]:
temp = spark.sql("SELECT userid, account_language FROM tweets_user_sql WHERE LENGTH(account_language)>5")
print("Number of rows with inconsistent account_language: " + str(temp.count()))
temp.show()

Number of rows with inconsistent account_language: 6
+--------------------+--------------------+
|              userid|    account_language|
+--------------------+--------------------+
|02b81295dbf8951d1...|          2016-01-13|
|          1240007161|          2013-03-03|
|8e77873eecf19db8d...|          2017-03-21|
|943154a86aa64a498...| islami bilgilər ...|
|bac526884ab0d54de...|          2017-03-02|
|bc1f64b72afcf37d0...|          2017-02-04|
+--------------------+--------------------+



In [22]:
# tweets_text_df.createOrReplaceTempView("tweets_text_sql")
# temp = spark.sql("SELECT U.userid, account_language, tweet_language FROM tweets_user_sql U, tweets_stats_sql S, tweets_text_sql T WHERE LENGTH(account_language)>5 AND U.userid=S.userid AND S.tweetid=T.tweetid")
# temp.dropDuplicates(['userid', 'tweet_language']).show()

In [23]:
tweets_user_df = tweets_user_df.withColumn('account_language', when(length(col('account_language'))>5, 'und').otherwise(col('account_language')))

In [24]:
tweets_user_df.createOrReplaceTempView("tweets_user_sql")

In [25]:
print("Number of rows with inconsistent account_language: " + str(spark.sql("SELECT userid, account_language FROM tweets_user_sql WHERE LENGTH(account_language)>5").count()))

Number of rows with inconsistent account_language: 0


We then split this dataframe into two:
* **anonymized_user_df** contains all the users that are anonymized.
* **exposed_user_df** contains all the other users.

This allows us to drop two columns for the anonymized users, which are a majority.

In [26]:
anonymized_user_df = spark.sql("SELECT * FROM tweets_user_sql WHERE userid=user_display_name")
exposed_user_df = spark.sql("SELECT * FROM tweets_user_sql WHERE NOT userid=user_display_name")

anonymized_user_df = anonymized_user_df.drop('user_display_name', 'user_screen_name')

print(tweets_user_df.count(), anonymized_user_df.count(), exposed_user_df.count())

4327 4118 209


### Summary

We have generated 4 main dataframes from the data files: *tweets_text_df*, *tweets_stats_df*, *tweets_meta_df*, and *tweets_user_df*. We then cleaned the inconsistent values and split those dataframes into smaller ones when possible and useful. Our data is now organised as follows:

* **tweets_text_df**: all the contents from the tweets, with an indication of their language.
* **tweets_stats_df**: 
    * **retweets_df**: all the information about retweets.
    * **replies_df**: all the information about replies.
    * **normal_tweets_df**: all the information about the other tweets.
* **tweets_meta_df**: all the meta information corresponding to each tweets (minus the latitude/longitude).
* **tweets_user_df**:
    * **anonymized_user_df**: all the information about anonymized users.
    * **exposed_user_df**: all the information about users who are not anonymized.
    